# Parameters

In [1]:
NUM_EPOCHS = 10
BOW = "tf_idf"
NETWORK = "ffnn"
NROWS = 100

# RNN
HIDDEN_SIZE = 10
NUM_LAYERS = 1

LEARNING_RATE = 0.001

BATCH_SIZE = 50

LOAD_PATH = None
SAVE_PATH = "OverwriteMe.pt"

## Import the necessary functions and libraries

In [2]:
from BoW import TF_IDF, getCountVector
from Preprocessing import getDataFrameFromData
from rnn import RNN
from networkFunctions import train, test
from ffnn import FFNN
from torch.utils.data import DataLoader
from dataSet import CustomDataDataSet

import torch 
import torch.nn as nn

## Read Data

In [3]:
train_df, train_labels = getDataFrameFromData("Archive/arxiv_train.csv", nrows=NROWS)
test_df, test_labels = getDataFrameFromData("Archive/arxiv_test.csv", nrows=NROWS)

## Calculate BoW and create dataloader

In [4]:
if BOW == "tf_idf":
    train_tensors, vocabulary = TF_IDF(train_df)
    test_tensors, _ = TF_IDF(test_df, vocabulary=vocabulary)
elif BOW == "countVector":
    train_tensors, vocabulary = getCountVector(train_df)
    test_tensors, _ = getCountVector(test_df, vocabulary)
else:
    raise SyntaxError(f"Ugyldig bow: {BOW}")


train_dataset = CustomDataDataSet(train_tensors, train_labels)
test_dataset = CustomDataDataSet(test_tensors, test_labels)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

c:\UIS\DAT550\dat550-prosjekt\BoW.py:26: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:620.)
  tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))


## Create Network Model

In [5]:
if NETWORK == "ffnn":
    model = FFNN(train_tensors.shape[1])
elif NETWORK == "rnn":
    model = RNN(train_tensors.shape[1], HIDDEN_SIZE, NUM_LAYERS)

## Load state dict if desired

In [6]:
if LOAD_PATH is not None:
	try:
		model.load_state_dict(torch.load(LOAD_PATH))
		print("Loaded model state")
	except FileNotFoundError:
		raise SyntaxError(f"Fant ingen fil: {LOAD_PATH}")

# Run testing. Train and save parameters if not pre-trained

In [7]:
loss_function = nn.CrossEntropyLoss()

if LOAD_PATH is None:
    optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for i in range(NUM_EPOCHS):
        train(train_dataloader, model, optimiser, loss_function)
        test(test_dataloader, model, loss_function)
    
    torch.save(model.state_dict(), SAVE_PATH)

else:
    for i in range(NUM_EPOCHS):
        test(test_dataloader, model, loss_function)

RuntimeError: Batches of sparse tensors are not currently supported by the default collate_fn; please provide a custom collate_fn to handle them appropriately.